<img src="dcic.png">

# Mapping Inequality in Baltimore

## Introduction 

After the 1929 Stock Market Crash, thousands of Americans were in danger of home foreclosure. To address this problem and stabilize the market, The Home Owners Loan Coperation was created under the Rooselvelt Administration to asses credibility of families in order to decide to refinance their homes.

- Redlining is the process 

- In particular Baltimore,

<img src="baltimoremap.png">

- A - 
- B - 
- C - 
- D - 

Learn more the Mapping Inequality Project [here](https://dsl.richmond.edu/panorama/redlining/#loc=4/36.71/-96.93&opacity=0.8)

- This data set
- Learning Objectives 

## 1. Aquiring Data

- something about json to csv, how we obtain our csv , rawdata

### Importing the Data and Creating a Data Frame

In [50]:
# Loads the Pandas library 
import pandas as pd

# Creates data frame (df) by reading in the Baltimore csv
df = pd.read_csv("AD_Data_BaltimoreProject.csv")
df.head()

,Form,State,City,Security_Grade,Area_Number,Terrain_Description,Favorable_Influences,Detrimental_Influences,INHABITANTS_Type,INHABITANTS_Annual_Income,...,INHABITANTS_Population_Increase,INHABITANTS_Population_Decrease,INHABITANTS_Population_Static,BUILDINGS_Types,BUILDINGS_Construction,BUILDINGS_Age,BUILDINGS_Repair,Ten_Fifteen_Desirability,Remarks,Date
0,NS FORM-8 6-1-37,Maryland,Baltimore,A,2,Rolling,Fairly new suburban area of homogeneous charac...,None,Substantial Middle Class,"$3000 - 5,000",...,Fast,NaN,NaN,Detached an row houses,Brick and frame,1 to 10 years,Good,Upward,A recent development with much room for expans...,"May 4,1937"
1,NS FORM-8 6-1-37,Maryland,Baltimore,A,1,Undulating,Very nicely planned residential area of medium...,None,"Executives, Professional Men",over $5000,...,Moderately Fast,NaN,NaN,Single family detached,Brick and Stone,12 years,Very good,Upward,Mostly fee properties. A few homes valued at $...,"May 4,1937"
2,NS FORM-8 6-1-37,Maryland,Baltimore,A,3,Rolling,Good residential area. Well planned.,Distance to City,"Executives, Professional Men",3500 - 7000,...,Moderately Fast,NaN,NaN,One family detached,"Brick, Stone, and Frame",1 to 20 years,Good to excellent,Upward,Principally fee property. This section lies in...,"May 4,1937"
3,NS FORM-8 6-1-37,Maryland,Baltimore,A,4,Level,Well planned development of fairly,None,Professional and Executives,over $5000,...,Slowly,NaN,NaN,One family,"Brick, Stone, and Frame",10 years,Good,Upward,All fee property,"May 4,1937"
4,NS FORM-8 6-1-37,Maryland,Baltimore,A,5,Undulating,"Desirable residential section. Good quality, m...",None,"Executives, Professional Men","$3,500 - $10,000",...,Moderately Fast,NaN,NaN,One family detached,Brick and Stone,1 to 20 years,Good,Upward,Merridale only recently developed. Prices do n...,NaN


- note on refering to the rows and columns - entities/attributes , observatons/variables

- form picture and corresponding attributes 

In [6]:
# Lists all the columns of the data frame
df.dtypes

Form                                object
State                               object
City                                object
Security_Grade                      object
Area_Number                        float64
Terrain_Description                 object
Favorable_Influences                object
Detrimental_Influences              object
INHABITANTS_Type                    object
INHABITANTS_Annual_Income           object
INHABITANTS_Foreignborn             object
INHABITANTS_F                      float64
INHABITANTS_Negro                   object
INHABITANTS_N                       object
INHABITANTS_Infiltration            object
INHABITANTS_Relief_Families         object
INHABITANTS_Population_Increase     object
INHABITANTS_Population_Decrease     object
INHABITANTS_Population_Static       object
BUILDINGS_Types                     object
BUILDINGS_Construction              object
BUILDINGS_Age                       object
BUILDINGS_Repair                    object
Ten_Fifteen

## 2. Data Cleaning 

### Handling Similar Text Values 

- Text

In [24]:
df.INHABITANTS_Foreignborn

0           No
1         None
2         None
3         None
4         None
5          NaN
6           No
7           No
8           No
9        Small
10    Very few
11          No
12          No
13          No
14          No
15     Mixture
16        None
17         Few
18          No
19          No
20        None
21        None
22          No
23          No
24          No
25         NaN
26         NaN
27         NaN
28          No
29       Small
30        None
31          No
32    Mixture 
33     Mixture
34     Mixture
35          No
36     Mixture
37     Mixture
38     Mixture
39     Mixture
40     Mixture
41     Nominal
42         NaN
43         NaN
44     Nominal
45         NaN
46         NaN
47         NaN
48         NaN
49         NaN
50    Italians
51      Polish
52     Mixture
53     Mixture
54     Mixture
55     Mixture
56     Mixture
Name: INHABITANTS_Foreignborn, dtype: object

In [31]:
# Replaces the values of 'No' with 'None'
df['INHABITANTS_Foreignborn'] = df['INHABITANTS_Foreignborn'].replace('No', 'None')

# Replaces all other values with 'Yes'
for value in df['INHABITANTS_Foreignborn']:
    if value != 'None':
        df['INHABITANTS_Foreignborn'] = df['INHABITANTS_Foreignborn'].replace(value, 'Yes')

0     None
1     None
2     None
3     None
4     None
5      Yes
6     None
7     None
8     None
9      Yes
10     Yes
11    None
12    None
13    None
14    None
15     Yes
16    None
17     Yes
18    None
19    None
20    None
21    None
22    None
23    None
24    None
25     Yes
26     Yes
27     Yes
28    None
29     Yes
30    None
31    None
32     Yes
33     Yes
34     Yes
35    None
36     Yes
37     Yes
38     Yes
39     Yes
40     Yes
41     Yes
42     Yes
43     Yes
44     Yes
45     Yes
46     Yes
47     Yes
48     Yes
49     Yes
50     Yes
51     Yes
52     Yes
53     Yes
54     Yes
55     Yes
56     Yes
Name: INHABITANTS_Foreignborn, dtype: object

In [5]:
no_words = ['No.','no', 'none', 'No', 'No.']

for word in no_words:
    df['Detrimental_Influences'] = df['Detrimental_Influences'].replace( word , 'None')

In [6]:
df.Detrimental_Influences

0                                                  None
1                                                  None
2                                      Distance to City
3                                                  None
4                                                  None
5                                                   NaN
6                                                  None
7                                                  None
8             Few streets of property in poor condition
9                                                  None
10                                                 None
11                              Built on filled ground.
12                                                 None
13                                                 None
14                           Distance to center of city
15                                                 None
16                                                 None
17        Distances to commercial area. Narrow s

- Note on functions

In [ ]:
for word in words_to_replace:
    df['INHABITANTS_Negro'] = df['INHABITANTS_Negro'].replace( word , 'None')
    
for word in words_to_replace:
    df['INHABITANTS_Infiltration '] = df['INHABITANTS_Infiltration'].replace( word , 'None')
    

In [7]:
small_words = ['Nominal','nominal', 'Small','small','Minimum', 'minimum','minimal', 'Very few']

for word in small_words:
    df ['INHABITANTS_Negro'] = df['INHABITANTS_Negro'].replace( word , 'Few')

for word in small_words:
    df ['INHABITANTS_Foreignborn'] = df['INHABITANTS_Foreignborn'].replace( word , 'Few')

In [8]:
def replace_values(attribute, new_word, words_to_replace):
    for word in words_to_replace:
        df[attribute] = df[attribute].replace(word, new_word) 

# example run
replace_values('INHABITANTS_Negro', 'None', no_words)

In [9]:
df.INHABITANTS_Negro

0     None
1     None
2     None
3     None
4     None
5      NaN
6     None
7     None
8     None
9     None
10    None
11    None
12    None
13    None
14    None
15    None
16    None
17    None
18     Yes
19    None
20    None
21    None
22    None
23    None
24    None
25     NaN
26     NaN
27     NaN
28     Few
29    None
      ... 
31    None
32     Yes
33     Yes
34    None
35    None
36     Yes
37    None
38     Yes
39    None
40     Yes
41    None
42     NaN
43     NaN
44     NaN
45     NaN
46     NaN
47     NaN
48     NaN
49     NaN
50     Yes
51     Yes
52     Yes
53     Yes
54     Yes
55     Yes
56     NaN
57     Yes
58     NaN
59     Yes
60     NaN
Name: INHABITANTS_Negro, dtype: object

### Encoding Categorical Data

- Note categorical degrees 

In [ ]:
# Will define degrees of speed as very slowly, slowly, moderately, moderately fast, fast

# Defines values to replace in the table 
very_slowly = ['Very Slowly', 'very slowly', 'very Slowly']
slowly = ['slowly', 'fairly slowly']
moderately = ['moderately']
moderately_fast = ['Moderately Fast', 'moderately Fast', 'moderately fast']
fast = ['very fast', 'Very Fast', 'fast']

# Applies function to obtain our new values 
replace_values('INHABITANTS_Population_Increase', 'Very slowly', very_slowly)
replace_values('INHABITANTS_Population_Increase', 'Slowly', slowly)
replace_values('INHABITANTS_Population_Increase', 'Moderately', moderately )
replace_values('INHABITANTS_Population_Increase', 'Moderately fast', moderately_fast)
replace_values('INHABITANTS_Population_Increase', 'Moderately fast', moderately_fast)
replace_values('INHABITANTS_Population_Increase', 'Fast', fast)


In [11]:
df.INHABITANTS_Population_Increase

0               Fast 
1     Moderately Fast
2     Moderately Fast
3              Slowly
4     Moderately Fast
5                 NaN
6                 NaN
7              Slowly
8     Moderately Fast
9     Moderately Fast
10                NaN
11             Slowly
12             Slowly
13         Moderately
14             Slowly
15              Fast 
16    Moderately fast
17         Moderately
18             Slowly
19             Slowly
20                NaN
21             Slowly
22              Fast 
23             Slowly
24                NaN
25                NaN
26                NaN
27                NaN
28                NaN
29             Slowly
           ...       
31                NaN
32        Very slowly
33        Fairly fast
34                NaN
35             Slowly
36                NaN
37             Slowly
38                NaN
39                NaN
40                NaN
41                NaN
42                NaN
43                NaN
44                NaN
45        

- Note on categorical to discrete by assigning values 

In [25]:
# Creates a new column in the data frame called 'Population_Increase_rank' and assigns default value as empty string
df.insert(17, 'Population_Increase_rank', " ")

In [26]:
# Does not work - probably need to duplicate the entire column instead and then replace values 
for value in df['INHABITANTS_Population_Increase']:
    if value == 'Very slowly':
     replace_values('Population_Increase_rank', '1', "")        

In [28]:
df.Population_Increase_rank

0      
1      
2      
3      
4      
5      
6      
7      
8      
9      
10     
11     
12     
13     
14     
15     
16     
17     
18     
19     
20     
21     
22     
23     
24     
25     
26     
27     
28     
29     
     ..
31     
32     
33     
34     
35     
36     
37     
38     
39     
40     
41     
42     
43     
44     
45     
46     
47     
48     
49     
50     
51     
52     
53     
54     
55     
56     
57     
58     
59     
60     
Name: Population_Increase_rank, dtype: object

- note on conditional statements 

### String Parsing 

- note on parsing text to extract data 

In [ ]:
# building_age

- note on regular expressions 

In [52]:
# Creates a new data frame that splits the values in the orginal data frame on the '-' character 
newdf = df["City"].str.split('-', n = 1, expand = True) 
newdf.head()

# Creates two new columns 
df['City_clean'] = newdf[0]
df['Suburb'] = newdf[1]

# removes the old column
df.drop(["City"], axis = 1, inplace = True) 

# renames as 'City'
df.rename(index=str, columns={"City_clean": "City"})

,Form,State,Security_Grade,Area_Number,Terrain_Description,Favorable_Influences,Detrimental_Influences,INHABITANTS_Type,INHABITANTS_Annual_Income,INHABITANTS_Foreignborn,...,INHABITANTS_Population_Static,BUILDINGS_Types,BUILDINGS_Construction,BUILDINGS_Age,BUILDINGS_Repair,Ten_Fifteen_Desirability,Remarks,Date,City,Suburb
0,NS FORM-8 6-1-37,Maryland,A,2,Rolling,Fairly new suburban area of homogeneous charac...,None,Substantial Middle Class,"$3000 - 5,000",No,...,NaN,Detached an row houses,Brick and frame,1 to 10 years,Good,Upward,A recent development with much room for expans...,"May 4,1937",Baltimore,None
1,NS FORM-8 6-1-37,Maryland,A,1,Undulating,Very nicely planned residential area of medium...,None,"Executives, Professional Men",over $5000,None,...,NaN,Single family detached,Brick and Stone,12 years,Very good,Upward,Mostly fee properties. A few homes valued at $...,"May 4,1937",Baltimore,None
2,NS FORM-8 6-1-37,Maryland,A,3,Rolling,Good residential area. Well planned.,Distance to City,"Executives, Professional Men",3500 - 7000,None,...,NaN,One family detached,"Brick, Stone, and Frame",1 to 20 years,Good to excellent,Upward,Principally fee property. This section lies in...,"May 4,1937",Baltimore,None
3,NS FORM-8 6-1-37,Maryland,A,4,Level,Well planned development of fairly,None,Professional and Executives,over $5000,None,...,NaN,One family,"Brick, Stone, and Frame",10 years,Good,Upward,All fee property,"May 4,1937",Baltimore,None
4,NS FORM-8 6-1-37,Maryland,A,5,Undulating,"Desirable residential section. Good quality, m...",None,"Executives, Professional Men","$3,500 - $10,000",None,...,NaN,One family detached,Brick and Stone,1 to 20 years,Good,Upward,Merridale only recently developed. Prices do n...,NaN,Baltimore,None
5,NS FORM-8 6-1-37,Maryland,A,6,Rolling,To be developed as high grade development,NaN,NaN,NaN,NaN,...,NaN,Single,Masonry,NaN,NaN,NaN,Owned and controlled by the Roland Park Compan...,NaN,Baltimore,Northwood
6,NS FORM-8 6-1-37,Maryland,B,1,Rolling to hilly,Desirable old residential section. Principally...,No.,"Executives, Professional Men",over $4000,No,...,Yes,One family detached,Frame,25 years,Good,Static,This section was completely built up in 1921. ...,NaN,Baltimore,None
7,NS FORM-8 6-1-37,Maryland,B,2,Rolling to hilly,"Restricted, desirable residential section of m...",None,"Executives, Professional Men",over $5000,No,...,NaN,One family detached,Frame and Brick,15 to 25 years,Good,Static to upward,Principally fee property.,"May 4,1937",Baltimore,None
8,NS FORM-8 6-1-37,Maryland,B,3,High level,Good residential section of fast moving property.,Few streets of property in poor condition,Business Men,$3000 and up,No,...,NaN,Single detached,Brick and Frame,15 years,Good,Static to upward,Section is very spotty with poor houses lying ...,"May 4,1937",Baltimore,None
9,NS FORM-8 6-1-37,Maryland,B,4,Rolling to level,Good suburban residential area of modest homes...,None,Business people,$2500 to $4000,Small,...,NaN,One family cottage,Frame,5 to 25 years,Good,Static to upward,Ground rent range $90 to $120. Part of the sec...,"May 4,1937",Baltimore,None


### Handling Dates 

- note seperating the date into three seperate columns because it contains three attributes not one

In [58]:
# need to split on multiple characters 
newdf = df['Date'].str.split(',', n = 1, expand = True)
newdf.head()

,0,1
0,May 4,1937
1,May 4,1937
2,May 4,1937
3,May 4,1937
4,NaN,None


- dates in pandas

## 3. Data Manipulation 

- 

### Income 

In [2]:
#Annual_income
# split the range into max and min 

# Create new column to store min and max values 
df['max_annual_income'] = " "

### Sorting 

In [ ]:
# Area Number 


### Grouping 

## 4. Data Analysis 

### Text Analysis 

## 5. Data Visualization 

<img src="redl.png">